In [1]:
import sys
sys.path.append("..")

## Load Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jtz18/skin-lesion")

c:\Users\chuanshaof\Projects\50.039-DL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chuanshaof\Projects\50.039-DL\venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for jtz18/skin-lesion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jtz18/skin-lesion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Seeding

In [3]:
import torch
import numpy as np
import random
import os

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

set_seed(42)

## Init Model

In [4]:
from src.segFormer import Segformer
from src.maluNet import MALUNet
from src.unet import UNET
from src.unetr import UNETR
# from src.unetr import UNETR

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model = UNETR(img_shape=(256, 256), num_heads= 3).to(device)

## Init Wandb

In [5]:
# Hyperparameters etc.
import wandb
LEARNING_RATE = 1e-5

DEVICE = device
BATCH_SIZE = 16

NUM_EPOCHS = 50

NUM_WORKERS = 0

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

PIN_MEMORY = True
LOAD_MODEL = False
CHECKPOINT_FILENAME = ""   # Model checkpoint filename if LOAD is True eg: checkpoints/checkpoint_9.pth.tar or None
CLASS = "task1"
MODEL = model.__class__.__name__

# Initialize a new run
run = wandb.init(project="50.039-DL", config={
    "learning_rate": LEARNING_RATE,
    "device": DEVICE,
    "batch_size": BATCH_SIZE,
    "num_epochs": NUM_EPOCHS,
    "num_workers": NUM_WORKERS,
    "image_height": IMAGE_HEIGHT,
    "image_width": IMAGE_WIDTH,
    "pin_memory": PIN_MEMORY,
    "load_model": LOAD_MODEL,
    "checkpoint_filename": CHECKPOINT_FILENAME,
    "class": CLASS,
    "Model": MODEL,
})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: forbidd3nzz (capstone21). Use `wandb login --relogin` to force relogin


# Define Training Function

In [6]:
from tqdm import tqdm
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            wandb.log({"Training Loss": loss})

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from src.utils.utils import *


train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


train_loader, val_loader, test_loader = get_loaders(
    dataset,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)

## Training Loop

In [8]:
start_epoch = 0
if LOAD_MODEL:
    checkpoint = torch.load(CHECKPOINT_FILENAME)
    model, optimizer, start_epoch = load_checkpoint(checkpoint, model, optimizer)



check_accuracy(val_loader, model, device=DEVICE)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"epoch: {epoch}")
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
        "epoch": epoch,
    }
    save_checkpoint(checkpoint, filename=f"checkpoint_{epoch}.pth.tar")

    # check accuracy
    check_accuracy(val_loader, model, device=DEVICE)

    # print some examples to a folder
    save_predictions_as_imgs(
        val_loader, model, folder=f"saved_images/{epoch}", device=DEVICE
    )

In [ ]:
run.finish()

In [ ]:
p = torch.zeros(5776, 3, 49, 49)
mask = torch.zeros(361, 49, 49)

mask = mask.unsqueeze(1)


p += mask

RuntimeError: The size of tensor a (5776) must match the size of tensor b (361) at non-singleton dimension 0